In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd ..
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive
/content/drive/MyDrive


In [4]:
import pickle

load the pickle file

In [5]:
model = pickle.load(open('knn_pickle_file', 'rb'))

In [6]:
! pip install Flask
!pip install pyngrok

In [7]:
!pip install flask-ngrok
!ngrok authtoken '2YStj1pM9wAHthoD7Uf0xu9eyJ5_6Bahc6v3st1Cq1s4WJDLr'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
!ngrok config add-authtoken '2YStj1pM9wAHthoD7Uf0xu9eyJ5_6Bahc6v3st1Cq1s4WJDLr'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


When the below code block is ran it asks for auth token input please input this: "2YStj1pM9wAHthoD7Uf0xu9eyJ5_6Bahc6v3st1Cq1s4WJDLr"

In [10]:
import getpass

from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect("22", "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel available, access with `ssh root@6.tcp.ngrok.io -p16893`


In [11]:
import os
import threading

from flask import Flask, jsonify,request,render_template
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
from werkzeug.datastructures import ImmutableMultiDict
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

app.config["BASE_URL"] = public_url
session = {}
# Define Flask routes
@app.route("/")
def index():
  ngrox_url = request.host_url
  return render_template('index.html', ngrox_url = ngrox_url)

@app.route("/predict", methods=["POST"])
def predict_price():
    ngrox_url = request.host_url
    print("Request")
    print(request.form)
    powerPS = request.form['powerPS']
    data = ImmutableMultiDict(request.form)
    print(data.get('fuelType'))

    kilometer = request.form['kilometer']
    model_id = request.form['model']
    abtest_id = request.form['abtest']
    vehicleType_id = request.form['vehicleType']
    gearbox_id = request.form['gearbox']
    notRepairedDamage_id = request.form['notRepairedDamage']
    features = int(powerPS), int(kilometer), int(model_id), int(abtest_id), int(vehicleType_id), int(gearbox_id), int(notRepairedDamage_id)
    print(features)
    session["powerPS"] = powerPS
    session["kilometer"] = kilometer
    session["model_id"] = request.form['model_name']
    session["abtest_id"] = request.form['abtest_name']
    session["vehicleType_id"] = request.form['vehicleType_name']
    session["gearbox_id"] = request.form['gearbox_name']
    session["notRepairedDamage_id"] = request.form['notRepairedDamage_name']
    try:
        prediction = model.predict([features])[0]
        print("prediction")
        print(prediction)
        return render_template('prediction.html', prediction = str(round(prediction)) + chr(36), session = session)

    except Exception as e:
        return jsonify({'error': str(e)})


 * ngrok tunnel "https://ceeb-35-245-177-15.ngrok-free.app" -> "http://127.0.0.1:5000"


After running the below code block click on the above ngrok tunnel which reidrects to a site and click in visit site then it redirects to the landing page

In [12]:
if __name__ == '__main__':
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on tcp://4.tcp.ngrok.io:10113
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:52:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:52:26] "GET /static/car_home_page.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:52:27] "GET /favicon.ico HTTP/1.1" 404 -


Request
ImmutableMultiDict([('vehicleType', '5'), ('vehicleType_name', 'Cabrio'), ('model', '8'), ('model_name', 'Scenic'), ('gearbox', '1'), ('gearbox_name', 'Automatic'), ('powerPS', '50'), ('kilometer', '500'), ('abtest', '1'), ('abtest_name', 'Control'), ('notRepairedDamage', '2'), ('notRepairedDamage_name', 'No')])
None
(50, 500, 8, 1, 5, 1, 2)
prediction
4474.833333333333


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:53:49] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:53:49] "GET /static/style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:53:49] "GET /static/prediction_car.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:53:49] "GET /static/graph-2.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:53:49] "GET /static/graph-3.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:53:49] "GET /static/graph-1.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Dec/2023 22:53:49] "GET /static/graph-4.jpeg HTTP/1.1" 200 -
